In [1]:
from spellsource.context import Context
from spellsource.behaviour import *
from spellsource.utils import simulate
from tqdm import tqdm

In [30]:
import pandas as pd
import random
import re
import collections

In [3]:
arena = pd.read_csv('cards.csv')
ctx = Context()

In [26]:
def generate_random_arena_deck(cards: pd.DataFrame, draft_class: str, deck_id: int):
    player_cards = cards[cards['Draft Class'] == draft_class]['Card Name']
    result = """### {}
# Class: {}
# Format: Standard
# Year of the Raven
#
""".format(deck_id, draft_class)
    for i in range(30):
        result += "# 1x (2) {}\n".format(random.choice(player_cards))
    return result

def get_deck_cards(deck: str):
    result = re.findall(r'# 1x \(2\) (.*)', deck)
    return result

In [22]:
desks = [generate_random_arena_deck(arena, 'Druid', i) for i in range(100)]

In [23]:
behaviour = 'PlayRandomBehaviour'
result = list(tqdm(simulate(
    behaviours=(behaviour, behaviour), 
    decks=desks,
    number=10,
    context=ctx)))

 79%|███████▉  | 3916/4950 [02:29<00:39, 26.17it/s]


In [29]:
result[0]

{'decks': ['44', '88'],
 'numberOfGames': 10,
 'results': [{'WIN_RATE': 0.7,
   'GAMES_WON': 7,
   'GAMES_LOST': 3,
   'DAMAGE_DEALT': 1181,
   'HEALING_DONE': 16,
   'MANA_SPENT': 693,
   'CARDS_PLAYED': 191,
   'TURNS_TAKEN': 143,
   'ARMOR_GAINED': 87,
   'CARDS_DRAWN': 152,
   'FATIGUE_DAMAGE': 10,
   'MINIONS_PLAYED': 351,
   'SPELLS_CAST': 36,
   'HERO_POWER_USED': 32},
  {'WIN_RATE': 0.3,
   'GAMES_WON': 3,
   'GAMES_LOST': 7,
   'DAMAGE_DEALT': 1144,
   'HEALING_DONE': 5,
   'MANA_SPENT': 693,
   'CARDS_PLAYED': 228,
   'TURNS_TAKEN': 138,
   'ARMOR_GAINED': 35,
   'CARDS_DRAWN': 138,
   'MINIONS_PLAYED': 372,
   'SPELLS_CAST': 16,
   'HERO_POWER_USED': 35,
   'WEAPONS_EQUIPPED': 6,
   'WEAPONS_PLAYED': 6,
   'CARDS_DISCARDED': 1}]}

In [37]:
win_prob = collections.Counter()
occur = collections.Counter()
for matchup in result:
    for index in range(2):
        deck = desks[int(matchup['decks'][index])]
        match_result = matchup['results'][index]
        for card in set(get_deck_cards(deck)):
            win_prob[card] += match_result['WIN_RATE']
            occur[card] += 1

In [45]:
win_rate = (pd.Series(win_prob) / pd.Series(occur)).sort_values(ascending=False)

In [55]:
win_rate['Ultimate Infestation']

0.6052272727272731